<a href="https://colab.research.google.com/github/HoChangLee98/my_study/blob/develop/PytorchTutorials/01quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

### 데이터 작업하기

In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

In [ ]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(f"Shape of X : {X.shape}")
  print(f"Shape of y : {y.shape} {y.dtype}")
  break

Shape of X : torch.Size([64, 1, 28, 28])
Shape of y : torch.Size([64]) torch.int64


### 모델 만들기

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 18)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=18, bias=True)
  )
)


### 모델 매개변수 최적화하기

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # 예측 오류 계산
    pred = model(X)
    loss = loss_fn(pred, y)

    # 역전파
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      # print(f"loss: {loss} [{current}/{size}]")
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1} \n ---------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1 
 ---------------------------
loss: 1.001147 [   64/60000]
loss: 1.040639 [ 6464/60000]
loss: 0.828346 [12864/60000]
loss: 0.986486 [19264/60000]
loss: 0.863750 [25664/60000]
loss: 0.889076 [32064/60000]
loss: 0.958520 [38464/60000]
loss: 0.894532 [44864/60000]
loss: 0.924740 [51264/60000]
loss: 0.877739 [57664/60000]
Test Error: 
 Accuracy: 67.7%, Avg loss: 0.888345 

Epoch 2 
 ---------------------------
loss: 0.922327 [   64/60000]
loss: 0.979418 [ 6464/60000]
loss: 0.755699 [12864/60000]
loss: 0.928600 [19264/60000]
loss: 0.810729 [25664/60000]
loss: 0.827854 [32064/60000]
loss: 0.910251 [38464/60000]
loss: 0.852935 [44864/60000]
loss: 0.873118 [51264/60000]
loss: 0.834847 [57664/60000]
Test Error: 
 Accuracy: 69.0%, Avg loss: 0.842105 

Epoch 3 
 ---------------------------
loss: 0.861172 [   64/60000]
loss: 0.931982 [ 6464/60000]
loss: 0.700449 [12864/60000]
loss: 0.885303 [19264/60000]
loss: 0.772218 [25664/60000]
loss: 0.781823 [32064/60000]
loss: 0.872728 [38464/60000]

### 모델 저장하기

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


### 모델 불러오기

In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot"
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  x = x.to(device)
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f"Predicted: '{predicted}', Actual: '{actual}'")

Predicted: 'Ankle boot', Actual: 'Ankle boot'
